# Kelompok 4 - Modul 7 PBF



# Tugas Kelompok

## Pardi_122450132
## Kartini_122450003
## Siti_122450006
## Elisabeth_122450123
## Ahmad_122450138

Langkah dalam menyelesaikan tugas ini adalah:

* Buatlah terlebih dahulu list dictionary yang kosong

* Buatlah looping untuk melakukan kalkulasi dengan fungsi Map atau Filter atau Reduce kemudian simpan ke dalam list dictionary yang sudah di definisikan

## Baca Ringkasan Data

In [ ]:
import pandas as pd

# Path ke file Excel
jalur_excel = '/content/911_Calls_for_Service_(Last_30_Days).csv'

# Baca file CSV (bukan Excel) menggunakan pd.read_csv
data_excel = pd.read_csv(jalur_excel)

# Tampilkan lima baris pertama data
print(data_excel.head())


           X          Y   incident_id agency               incident_address  \
0 -82.993989  42.410293  202306901627   DPD     Glenfield Ave & Gratiot Ave   
1 -83.052792  42.415266  202306901629   DPD            Luce St & Fenelon St   
2 -83.072103  42.368421  202306901630   DPD   W Baltimore St & Woodward Ave   
3 -83.226541  42.429892  202306901632   DPD          Avon Ave & W 7 Mile Rd   
4 -83.240073  42.409465  202306901633   DPD     Heyden St & Grand River Ave   

   zip_code priority callcode             calldescription  category  ...  \
0     48213        3   W4807   START OF SHIFT INFORMATION  STRTSHFT  ...   
1     48212        3   SA               SPECIAL ATTENTION  SPCL ATT  ...   
2     48202        3   SA               SPECIAL ATTENTION  SPCL ATT  ...   
3     48219        4   372040                 UDAA REPORT  UDAAREPT  ...   
4     48219        3   SA               SPECIAL ATTENTION  SPCL ATT  ...   

  traveltime totalresponsetime time_on_scene totaltime     neighborh

## Bagian 1: Model Populasi Polisi Detroit

Baca data dari file Laporan Polisi Detroit menggunakan modul csv dan terjemahkan data ke dalam daftar dictionary. Menggunakan Filter dengan fungsi lambda untuk mengecualikan dictionary (baris CSV) yang memiliki data yang hilang di kolom Zip, atau kolom Neighborhood. Dengan menggunakan fungsi lambda dan Reduce, hitung total waktu respons rata-rata, waktu pengiriman rata-rata, dan total waktu rata-rata untuk kepolisian Detroit.

In [ ]:
import csv
from functools import reduce

# Fungsi untuk membaca file CSV dan memfilter baris yang lengkap
def baca_csv_filter(nama_file):
    with open(nama_file, 'r') as file:
        pembaca = csv.DictReader(file)
        data = filter(lambda x: x['zip_code'] != '' and x['neighborhood'] != '', pembaca)
        return list(data)

# Menghitung total waktu respons rata-rata, waktu pengiriman rata-rata, dan total waktu rata-rata
def hitung_rerata(data):
    total_waktu_respons = 0
    total_waktu_pengiriman = 0
    total_waktu = 0
    jumlah_rekaman = 0

    for baris in data:
        try:
            total_waktu_respons += float(baris['totalresponsetime'])
            total_waktu_pengiriman += float(baris['traveltime'])
            total_waktu += float(baris['totaltime'])
            jumlah_rekaman += 1
        except ValueError:
            pass  # Tidak lakukan apa-apa jika terjadi kesalahan dalam konversi nilai ke float

    if jumlah_rekaman == 0:
        return {
            'neighborhood': 'DETROIT',  # Tambahkan kunci neighborhood dengan nilai 'DETROIT'
            'rerata_waktu_respons': 0,
            'rerata_waktu_pengiriman': 0,
            'rerata_total_waktu': 0
        }

    return {
        'neighborhood': 'DETROIT',  # Tambahkan kunci neighborhood dengan nilai 'DETROIT'
        'rerata_waktu_respons': total_waktu_respons / jumlah_rekaman,
        'rerata_waktu_pengiriman': total_waktu_pengiriman / jumlah_rekaman,
        'rerata_total_waktu': total_waktu / jumlah_rekaman
    }



# Path file CSV
path_csv = "/content/911_Calls_for_Service_(Last_30_Days).csv"

# Baca file CSV dan filter data
data_terfilter = baca_csv_filter(path_csv)

# Hitung rata-rata waktu respons, waktu pengiriman, dan total waktu
hasil_part1 = hitung_rerata(data_terfilter)
print(hasil_part1)


{'neighborhood': 'DETROIT', 'rerata_waktu_respons': 8.336226605030683, 'rerata_waktu_pengiriman': 2.3334066254288293, 'rerata_total_waktu': 29.530862153463513}


## Bagian 2: Modelkan Neighborhood Samples

Menggunakan fungsi lambda dan map, atau lambda dan Filter, bagi list dictionary menjadi list dictionary yang lebih kecil yang dipisahkan oleh neighborhood. Dengan menggunakan lambda dan Reduce, temukan total waktu respons rata-rata untuk setiap neighborhood, waktu pengiriman rata-rata untuk setiap neighborhood, dan total waktu rata-rata untuk setiap neighborhood dan simpanlah ke dalam list dictionary. Tambahkan item dictioanary untuk menyertakan data populasi untuk semua Detroit dalam daftar gabungan Anda.

In [ ]:
from functools import reduce

# Mengelompokkan data berdasarkan daerah (neighborhood)
def kelompokkan_berdasarkan_daerah(data):
    data_terkelompok = {}
    for baris in data:
        daerah = baris['neighborhood']
        if daerah not in data_terkelompok:
            data_terkelompok[daerah] = []
        data_terkelompok[daerah].append(baris)
    return data_terkelompok

# Menghitung rata-rata waktu respons, waktu perjalanan, dan total waktu untuk setiap daerah
def hitung_rata_rata_daerah(data):
    rata_rata = {}
    for daerah, catatan in data.items():
        if not catatan:
            rata_rata[daerah] = {
                'rata_rata_waktu_respons': 0,
                'rata_rata_waktu_perjalanan': 0,
                'rata_rata_waktu_total': 0,
                'populasi': 0
            }
        else:
            waktu_respons = [float(baris['totalresponsetime']) for baris in catatan if baris['totalresponsetime'].strip()]
            waktu_perjalanan = [float(baris['traveltime']) for baris in catatan if baris['traveltime'].strip()]
            waktu_total = [float(baris['totaltime']) for baris in catatan if baris['totaltime'].strip()]

            jumlah_catatan = len(waktu_respons)
            if jumlah_catatan > 0:
                total_waktu_respons = sum(waktu_respons)
                total_waktu_perjalanan = sum(waktu_perjalanan)
                total_waktu = sum(waktu_total)
                rata_rata[daerah] = {
                    'rata_rata_waktu_respons': total_waktu_respons / jumlah_catatan,
                    'rata_rata_waktu_perjalanan': total_waktu_perjalanan / jumlah_catatan,
                    'rata_rata_waktu_total': total_waktu / jumlah_catatan,
                    'populasi': len(catatan)  # Menambahkan populasi setiap daerah
                }
            else:
                rata_rata[daerah] = {
                    'rata_rata_waktu_respons': 0,
                    'rata_rata_waktu_perjalanan': 0,
                    'rata_rata_waktu_total': 0,
                    'populasi': len(catatan)
                }
    return rata_rata


# Fungsi untuk membaca file CSV dan memfilter baris yang lengkap
def baca_csv_saring(nama_file):
    with open(nama_file, 'r') as file:
        pembaca = csv.DictReader(file)
        data = filter(lambda x: x['zip_code'] != '' and x['neighborhood'] != '', pembaca)
        return list(data)

# Bagian 2 - Kelompokkan data berdasarkan daerah
data_terkelompok = kelompokkan_berdasarkan_daerah(data_terfilter)

# Bagian 3 - Hitung rata-rata waktu respons, waktu perjalanan, dan total waktu untuk setiap daerah
rata_rata_daerah = hitung_rata_rata_daerah(data_terkelompok)

# Tambahkan item dictionary untuk menyertakan data populasi untuk semua Detroit dalam daftar gabungan
total_populasi = len(data_terfilter)
for daerah in rata_rata_daerah:
    rata_rata_daerah[daerah]['total_populasi'] = total_populasi

print(rata_rata_daerah)


{'Eden Gardens': {'rata_rata_waktu_respons': 13.989189189189194, 'rata_rata_waktu_perjalanan': 2.247747747747748, 'rata_rata_waktu_total': 41.68603603603606, 'populasi': 234, 'total_populasi': 78442}, 'Buffalo Charles': {'rata_rata_waktu_respons': 15.773099415204678, 'rata_rata_waktu_perjalanan': 3.570760233918129, 'rata_rata_waktu_total': 45.3017543859649, 'populasi': 177, 'total_populasi': 78442}, 'New Center': {'rata_rata_waktu_respons': 1.5898351648351647, 'rata_rata_waktu_perjalanan': 0.5021978021978022, 'rata_rata_waktu_total': 15.547390109890097, 'populasi': 745, 'total_populasi': 78442}, "O'Hair Park": {'rata_rata_waktu_respons': 15.277704918032784, 'rata_rata_waktu_perjalanan': 4.552459016393445, 'rata_rata_waktu_total': 47.89868852459019, 'populasi': 325, 'total_populasi': 78442}, 'Miller Grove': {'rata_rata_waktu_respons': 11.734272300469481, 'rata_rata_waktu_perjalanan': 2.1553990610328633, 'rata_rata_waktu_total': 33.317840375586854, 'populasi': 218, 'total_populasi': 7844

## Bagian 3: Buat file Output JSON

Menggunakan modul JSON, format list dictionary Anda sebagai JSON

In [ ]:
import json

# Bagian 3: Buat file Output JSON
def simpan_ke_json(data_part1, data_daerah, nama_file):
    with open(nama_file, 'w') as berkas_json:
        json.dump({'hasil_part1': data_part1, 'rata_rata_daerah': data_daerah}, berkas_json, indent=4)

# Path untuk menyimpan file JSON
path_output_json = "output.json"

# Simpan data ke dalam file JSON
simpan_ke_json(hasil_part1, rata_rata_daerah, path_output_json)

print("Data telah disimpan dalam file JSON:", path_output_json)


Data telah disimpan dalam file JSON: output.json
